<center>Национальный исследовательский университет ИТМО<br/>Факультет информационных технологий и программирования<br/>Прикладная математика и информатика</center>

## <center>Математическая статистика</center>
### <center>Отчёт по лабораторной работе №4</center>

<div style="text-align: right"><b>Работу выполнили:</b><br/>Белоцерковченко Андрей M3337<br/>Смирнов Андрей M3337<br/</div>

<center>Санкт-Петербург<br/>2023</center>

# Вариант 2

In [110]:
import numpy as np
from scipy.special import comb
from scipy.stats import norm
from scipy.stats import rankdata
from scipy.stats import chi2
import pandas as pd
import math
df = pd.read_csv('exams_dataset.csv')
alpha = 0.05

## Задача 2
В задаче мы хотим проверить, что выборки результатов по математике и письменной части имеют одинаковое распределение. Более формально: $$H_0: F_X = F_Y$$ $$H_1: F_X \neq F_Y $$ Здесь и далее уровень значимости $\alpha = 0.05$

### Критерий знаков
Пусть результаты по математике будут равны $X_1, X_2, X_3 ... X_n$, а по письменной части $Y_1, Y_2, Y_3 ... Y_n$. Возьмём последовательность $Z$, такую что $sign(X_1 - Y_1), sign(X_2 - Y_2), sign(X_3 - Y_3) ... sign(X_n - Y_n)$. Выкинем из рассмотрения все нули (случаи, когда $X_i = Y_i$). Мы хотим проверить, правда ли, что $P(Z > 0) = 0.5$.
Переформулируем гипотезы: $$H_0: p = \cfrac{1}{2}$$ $$H_1: p \neq \cfrac{1}{2}$$
Пусть $w$ - количество $Z_i = 1$. Если $H_0$ выполняется, то $w \sim B(n, 0.5)$ А дальше просто проверяем, что $b = \cfrac{\sum\limits_{i = 0}^w C_n^i}{2^n}$ попадает в критическую область $(\cfrac{\alpha}{2}, 1 - \cfrac{\alpha}{2})$, где $\alpha$ - уровень значимости равный $0.05$

In [111]:
sign = lambda x: math.copysign(1, x)
test = (df['math score'] - df['writing score']).apply(sign)
w = test.gt(0).sum()
cSum = 0
for i in range(0, w + 1):
    cSum += comb(1000, i, exact=True)

b = cSum / (2 ** 1000)
print("Sign stat:", b)
print("Check if stat in critical region:", (alpha / 2) < b < 1 - (alpha / 2))

Sign stat: 0.0024308681959370596
Check if stat in critical region: False


## Критерий Ван дер Вардена

Определим величину $A_{ij}$: $$A_{ij} = \Phi^{-1}(\cfrac{R(X_{ij})}{N + 1})$$ где $R(X_{ij})$ - ранг $i$-ого элемента $j$-ой выборки в объединённой выборке, N - размер объединённой выборки. Также введём величины $\overline{A_j}$ и $s^2$: $$\overline{A_j} = \cfrac{1}{n_j}\sum\limits_{i = 1}^{n_j}A_{ij}$$ $$s^2 = \cfrac{1}{N - 1}\sum\limits_{j = 1}^k\sum\limits_{i = 1}^{n_j}A_{ij}^2$$
Тогда статистика считается по формуле $$T = \cfrac{1}{s^2}\sum\limits_{j = 1}n_j\overline{A_j}^2$$
Поскольку тест Ван дер Вардена описан для $k$ выборок - в оригинале гипотеза $H_1$ звучит так: "хотя бы одна из выборок имеет тенденцию давать более крупные значения, чем другие" Но поскольку в нашем случае мы проверяем однородность только двух выборок - можем оставить $H_1$ без изменения.

При этом нуль-гипотеза отвергается, если статистика попадёт в промежуток $T > \chi^2_{\alpha, k - 1}$

In [112]:
def count_mean_rank(f, s, d):
    n = len(f)
    m = len(s)
    k = n + m + 1
    R = 0
    for x in f:
        R += norm.ppf(d[x] / k)
        
    return R / n

def count_variance(f, s):
    n = len(f)
    m = len(s)
    
    R = 0
    for x in f:
        R += x
    for y in s:
        R += y
        
    return R / (n + m - 1)

a = df['math score']
b = df['writing score']
u = np.concatenate((a, b))
number_to_rank = dict(zip(u, rankdata(u)))
stat = ((count_mean_rank(a, b, number_to_rank) ** 2) * len(a) + 
        (count_mean_rank(b, a, number_to_rank) ** 2) * len(b)) / count_variance(a, b)
print("Van der Waerden stat:", stat)
print("Check if stat in critical region:", chi2.ppf(alpha, 1) < stat)

Van der Waerden stat: 0.10906549996717955
Check if stat in critical region: True


Таким образом, оба критерия показали неоднородность результатов экзамена по математике и письменной части.  